In [22]:
# import os
# from pathlib import Path
# # Create a symlink to the vbt.ini file in the parent directory
# ini_file_path = Path.cwd().parent.parent / "vbt.ini"
# link = Path.cwd() / "vbt.ini"
# # Create the symlink
# if not link.exists():  # Check if the symlink doesn't already exist
#     os.symlink(ini_file_path, link)

from vectorbtpro import *
import pandas as pd

vbt.settings.set_theme("dark")
vbt.settings.plotting["layout"]["width"] = 800
vbt.settings.plotting['layout']['height'] = 200
vbt.settings.plotting.use_resampler = True # Need to pip install https://github.com/predict-idlab/plotly-resampler

pd.set_option('display.max_rows', None)

# Type out the version of vectorbtpro Note for this instance I'm using 2024.2.22 note it has a different way of importing Vectorbt see release notes https://vectorbt.pro/pvt_321460c7/getting-started/release-notes/
vbt.__version__

'2024.2.22'

## Create some additional metrics for VBT
In the next cell we create some additional metrics that I wanted to track for portfolios, ie. `capital_weighted_time_in_market` which is helpful for analyzing the impact of leverage and time in market, ie if you only invest 1% of your acccount 100% of the time then your `time_in_market` would be 100% but that's really misleading because your total max exposure was only 1%, and vice versa, if you invested 400% of your account only 25% of the time your `time_in_market` would be 25% but your `capital_weighted_time_in_market` would be 100%.

In [8]:
def calc_capital_weighted_time_in_market(portfolio):
    portfolio_trades = portfolio.trades.records_readable
    # Calculate trade duration and convert to seconds
    trade_duration_seconds = (portfolio_trades['Exit Index'] - portfolio_trades['Entry Index']).dt.total_seconds()
    capital_invested = portfolio_trades.Size * portfolio_trades['Avg Entry Price']
    weighted_time = (trade_duration_seconds * capital_invested).sum()
    # Calculate total time in seconds
    total_time_seconds = (portfolio.wrapper.index[-1] - portfolio.wrapper.index[0]).total_seconds()
    capital_weighted_time_pct = (weighted_time / (total_time_seconds * portfolio.value.mean())) * 100
    return capital_weighted_time_pct

# print(f'Capital Weighted Time Invested [%]: {calc_capital_weighted_time_in_market(pf):.2f}%')

# Example of how to create a custom metric
# https://vectorbt.dev/api/portfolio/base/#custom-metrics
# Define your custom metrics
max_winning_streak = (
    'max_winning_streak',
    dict(
        title='Max Winning Streak',
        calc_func='trades.winning_streak.max'
    )
)

max_losing_streak = (
    'max_losing_streak',
    dict(
        title='Max Losing Streak',
        calc_func='trades.losing_streak.max'
    )
)

capital_weighted_time_exposure = (
    'capital_weighted_time_exposure',
    dict(
        title='Capital Weighted Time Exposure [%]',
        calc_func=lambda self, group_by:
        calc_capital_weighted_time_in_market(self)
    )
)

custom_metrics_dict = {
    'max_winning_streak': max_winning_streak,
    'max_losing_streak': max_losing_streak,
    'capital_weighted_time_exposure': capital_weighted_time_exposure
}

# Retrieve the default metrics and convert them to a dictionary
default_metrics_dict = dict(vbt.Portfolio.metrics)


# Reorder metrics according to desired_order
desired_order = [
    'start', 'end', 'period', 'start_value', 'min_value', 'max_value', 
    'end_value', 'cash_deposits', 'cash_earnings', 'total_return', 
    'bm_return', 'total_time_exposure', 'capital_weighted_time_exposure', 
    'max_gross_exposure', 'max_dd', 'max_dd_duration', 'total_orders', 
    'total_fees_paid', 'total_trades', 'win_rate', 'max_winning_streak', 
    'max_losing_streak', 'best_trade', 'worst_trade', 'avg_winning_trade', 
    'avg_losing_trade', 'avg_winning_trade_duration', 'avg_losing_trade_duration', 
    'profit_factor', 'expectancy', 'sharpe_ratio', 'calmar_ratio', 
    'omega_ratio', 'sortino_ratio'
]

# Reorder metrics according to desired_order
ordered_metrics = []
for metric_name in desired_order:
    if metric_name in custom_metrics_dict:
        # Add custom metric
        ordered_metrics.append(custom_metrics_dict[metric_name])
    elif metric_name in default_metrics_dict:
        # Add default metric
        ordered_metrics.append(metric_name)
    else:
        print(f"Warning: Metric '{metric_name}' not found.")

# Now call the stats method with the ordered metrics
# pf.stats(metrics=ordered_metrics)




## Import the trades and prep them for analysis

In [ ]:
# filename = 'trade history - 12.2023.xlsx'
# filename = 'data/Crypto-to-the-moon-orders.csv'
filename = 'data/Average-Moon-Cypress-orders.csv'
# filename = 'data/fat_bear-orders.csv'
# filename = 'data/RealCryptoFox-orders.csv'
trades = pd.read_csv(filename)

trades['open_date'] = pd.to_datetime(trades['open_date'])
trades['close_date'] = pd.to_datetime(trades['closed_date'])
trades['symbol'] = trades['title'].str.extract(r'(\w+)')
# Change USDT to -USDT
trades['okx_symbol'] = trades['symbol'].str.replace('USDT', '-USDT')
trades['trade_type'] = trades['leverage'].apply(lambda x: 'open_long' if 'Long' in x else 'close_long' if 'Long' in x else 'open_short' if 'Short' in x else 'close_short')
trades = trades.drop(columns=['Unnamed: 0'])
# Function to clean and convert currency columns
def clean_currency_column(column):
    return pd.to_numeric(column.str.replace(',', '').str.replace(' USDT', ''), errors='coerce')

# Clean the currency columns
trades['entry_price'] = clean_currency_column(trades['entry_price'])
trades['pnl'] = clean_currency_column(trades['pnl'])
trades['fill_price'] = clean_currency_column(trades['fill_price'])

# Extracting the number of contracts
trades['num_contracts'] = trades['closed'].str.extract('(\d+,?\d*)').replace(',', '', regex=True).astype(int)

# Get a list of all unique symbols
symbols = trades['symbol'].unique()
okx_symbols = trades['okx_symbol'].unique()
# Look up the contract multiplier for each symbol from the exchange website https://www.okx.com/trade-market/info/swap
symbols_dict_contract_multiplier = {'ETH-USDT':0.1, 'BTC-USDT':0.01, 'PEOPLE-USDT':100, 'ORDI-USDT':0.1, 'SOL-USDT':1,
       'DOGE-USDT':1000, 'USTC-USDT':100, 'BNB-USDT':0.01}
trades['contract_multiplier'] = trades['okx_symbol'].map(symbols_dict_contract_multiplier)
trades['quantity'] = trades['num_contracts'] * trades['contract_multiplier']

# Now separate out the orders into open and closing orders and sort by date
# Open Orders
open_orders = trades[['open_date', 'title', 'direction', 'leverage', 'entry_price', 'symbol', 'okx_symbol', 'num_contracts', 'quantity']].copy()
open_orders.rename(columns={'open_date': 'date'}, inplace=True)
open_orders['price'] = open_orders['entry_price']
open_orders['trade_type'] = open_orders['leverage'].apply(lambda x: 'open_long' if 'Long' in x else 'open_short')

# Closing Orders
closing_orders = trades[['closed_date', 'title', 'direction', 'leverage', 'fill_price', 'pnl', 'pnl_percent', 'symbol', 'okx_symbol', 'num_contracts', 'quantity']].copy()
closing_orders.rename(columns={'closed_date': 'date', 'fill_price': 'close_price'}, inplace=True)
closing_orders['price'] = closing_orders['close_price']
closing_orders['trade_type'] = closing_orders['leverage'].apply(lambda x: 'close_long' if 'Long' in x else 'close_short')

# Combine the two dataframes
orders = pd.concat([open_orders, closing_orders])

# Convert date columns to datetime for sorting
orders['date'] = pd.to_datetime(orders['date'], errors='coerce')

# Sorting by date
orders = orders.sort_values(by='date')
orders.set_index('date', inplace=True)
# Localize the index to Central Standard Time (CST) first
orders.index = orders.index.tz_localize('America/Chicago')
# Then convert the timezone from CST to UTC
orders.index = orders.index.tz_convert('UTC')
# Revised approach to handle duplicate timestamps by adding milliseconds
def add_milliseconds_to_duplicates(df):
    # Create a new Series to hold adjusted timestamps
    adjusted_timestamps = []

    # Create a dictionary to track the count of each timestamp
    timestamp_count = {}

    # Iterate through each timestamp in the index
    for timestamp in df.index:
        # If the timestamp is not in the dictionary, add it with a count of 0
        if timestamp not in timestamp_count:
            timestamp_count[timestamp] = 0
            adjusted_timestamps.append(timestamp)
        else:
            # If the timestamp is already in the dictionary, increment the count
            timestamp_count[timestamp] += 1
            # Add milliseconds to the timestamp based on its count
            new_timestamp = timestamp + pd.Timedelta(milliseconds=timestamp_count[timestamp])
            adjusted_timestamps.append(new_timestamp)

    return pd.Series(adjusted_timestamps, index=df.index)

# # Apply the function to adjust the index
adjusted_index = add_milliseconds_to_duplicates(orders)
orders.index = adjusted_index
orders = orders.sort_index()
# If open-short and close-long then quantity should be negative else positive
orders['trade_direction'] = orders.apply(lambda x: -1 if (x['trade_type'] == 'open_short' or x['trade_type'] == 'close_long') else 1, axis=1)
orders['trade_quantity'] = orders['trade_direction'] * orders['quantity']

# display(symbols)
# display(okx_symbols)
# orders.tail(20)


# Download minutely data from Binance or load from pickle file


In [5]:
from datetime import datetime
min_date = trades['open_date'].min().floor('D')
max_date = trades['close_date'].max().ceil('D')

print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

# Pull data from binance for BTC and ETH
main_symbols = ['BTCUSDT', 'ETHUSDT']
# data = vbt.BinanceData.pull(main_symbols, start=min_date, end=max_date, timeframe='15T')
# data.save('price_data.pkl')
data = vbt.Data.load('price_data.pkl')
data['Close'].get().tail()

Minimum date: 2023-11-21 00:00:00
Maximum date: 2024-02-22 00:00:00


symbol,BTCUSDT,ETHUSDT
Open time,,
2024-02-21 22:45:00+00:00,51511.10,2934.20
2024-02-21 23:00:00+00:00,51624.85,2947.11
2024-02-21 23:15:00+00:00,51650.71,2951.49
2024-02-21 23:30:00+00:00,51694.02,2973.60
2024-02-21 23:45:00+00:00,51849.39,2967.90


## Calculate the historical performance

In [7]:
# Choose a symbol to analyze
symbol = 'BTCUSDT'

symbol_data = data['Close'].get(symbol=symbol)
symbol_orders = orders[orders['symbol'] == symbol]

# Identify duplicates in the index
symbol_orders.index[symbol_orders.index.duplicated()]
# Throw an error if there are duplicates
if symbol_orders.index.duplicated().any():
    raise ValueError('Duplicate timestamps found in the orders dataframe')

# Change Floor and Ceiling based on your preference to zoom in or out. This is just rounding the date to the nearest hour/day/etc.
min_date = symbol_orders.index.min().floor('D') 
max_date = symbol_orders.index.max().ceil('D')
print(f"Minimum date: {min_date}")
print(f"Maximum date: {max_date}")

unlevered_pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    init_cash = 'auto', # This sets the initial cash based on the largest trade
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)
print(f'Unlevered Portfolio Sim Sharpe Ratio: {unlevered_pf.sharpe_ratio}')

# Now Grab the auto calculated initial cash so you can add leverage to your portfolio
init_cash = unlevered_pf.init_cash 
leverage = 3 # This can be adjusted to your preference

pf = vbt.Portfolio.from_orders(
    close   = symbol_data.loc[min_date:max_date], # Note, here we are using the minutely data for ETH
    size    = symbol_orders['trade_quantity'],  
    price   = symbol_orders['price'],
    size_type = 'amount',
    # fixed_fees = trades['fees'],
    init_cash = init_cash/leverage,
    leverage=leverage,
    leverage_mode=vbt.pf_enums.LeverageMode.Eager,    
    freq = '15T',
)

display(pf.stats(metrics=ordered_metrics)) # This is where we incorporate our custom metrics
pf.plot().show()

Minimum date: 2023-11-21 00:00:00+00:00
Maximum date: 2024-02-21 00:00:00+00:00
Unlevered Portfolio Sim Sharpe Ratio: 1.4578722089055722


Start                                 2023-11-21 00:00:00+00:00
End                                   2024-02-21 00:00:00+00:00
Period                                         94 days 19:30:00
Start Value                                       237982.973333
Min Value                                         236856.993333
Max Value                                         290314.173333
End Value                                         264578.633333
Total Return [%]                                      11.175447
Benchmark Return [%]                                  39.679138
Total Time Exposure [%]                               38.321248
Capital Weighted Time Exposure [%]                    53.440896
Max Gross Exposure [%]                               293.753967
Max Drawdown [%]                                      10.924468
Max Drawdown Duration                          21 days 13:30:00
Total Orders                                                270
Total Fees Paid                         

# Some slick analysis tools in vbt


look at the trades visually

In [15]:
unlevered_pf.plot_trade_signals().show()

plot expanding metrics. "How have the trades behaved over time?"
Regular metrics such as MAE and MFE represent only the final point of each trade, but what if we would like to see their development during each trade? You can now analyze expanding trade metrics as DataFrames!

In [19]:
unlevered_pf.trades.plot_expanding_mae_returns(title='Worst Trades').show() # This is the maximum adverse excursion (MAE)
unlevered_pf.trades.plot_expanding_mfe_returns(title='Best Trades').show() # This is the maximum favorable excursion (MFE)

trade_history see a clean summary of all the trades
plot_mae and plot_mfe these show you the most painful (MAE) and most profitable moments while in a trade. Link to documentation [here](https://vectorbt.pro/pvt_321460c7/features/analysis/#mae-and-mfe)

In [20]:
display(unlevered_pf.trade_history.head())
unlevered_pf.trades.plot_mae_returns(title='Worst Trades').show()
unlevered_pf.trades.plot_mfe_returns(title='Best Trades').show()

,Order Id,Column,Index,Side,Size,Price,Fees,PnL,Return,Direction,Status,Entry Trade Id,Exit Trade Id,Position Id
0,0,0,2023-11-21 13:43:27+00:00,Sell,4.0,37071.6,0.0,708.8,0.004780,Short,Closed,0,-1,0
1,1,0,2023-11-21 14:11:19+00:00,Buy,4.0,36894.4,0.0,708.8,0.004780,Short,Closed,-1,0,0
2,2,0,2023-11-22 06:34:16+00:00,Buy,2.0,36490.0,0.0,24.0,0.000329,Long,Closed,1,-1,1
3,3,0,2023-11-22 12:31:41+00:00,Sell,2.0,36502.0,0.0,24.0,0.000329,Long,Closed,-1,1,1
4,4,0,2023-11-22 13:48:59+00:00,Buy,4.0,36574.7,0.0,-402.8,-0.002753,Long,Closed,2,-1,2


NameError: name 'unlevered_pf' is not defined